# Design Principles

## Encapsulate what Varies

> Identify the aspects of your application that vary and separate them from what stays the same.


### Encapsulation on a method level

In [16]:
from dataclasses import dataclass


@dataclass
class Item:
    price: int
    quantity: int


@dataclass
class Order:
    line_items: list[Item]
    country: str

    def get_order_total(self):
        total = sum([item.price * item.quantity for item in self.line_items])

        if self.country == "US":
            total += total * 0.07  # US sales tax
        elif self.country == "EU":
            total += total * 0.20  # European VAT

        return total


In [22]:
# SOLUTION
from dataclasses import dataclass


@dataclass
class Item:
    price: int
    quantity: int


@dataclass
class Order:
    line_items: list[Item]
    country: str

    def get_order_total(self):
        total = sum([item.price * item.quantity for item in self.line_items])
        total += total * self.get_tax_rate()
        return total


    def get_tax_rate(self):
            if self.country == "US":
                return 0.07  # US sales tax
            elif self.country == "EU":
                return 0.20  # European VAT
            else:
                return 0


In [21]:
# Unit Test
items = [Item(20, 10), Item(1, 100)]
order1 = Order(items, "US")
order2 = Order(items, "EU")
order3 = Order(items, "NZ")

assert order1.get_order_total() == 321
assert order2.get_order_total() == 360
assert order3.get_order_total() == 300

### Encapsulation on a class level


In [1]:
from collections import defaultdict
from dataclasses import dataclass


@dataclass
class Item:
    price: int
    quantity: int
    category: str


@dataclass
class Order:
    line_items: list[Item]
    country: str
    state: str = None

    def get_order_total(self):
        total = sum([item.price * item.quantity for item in self.line_items])
        total += total * self.get_tax_rate()
        return total

    def get_tax_rate(self):
        if self.country == "US":
            state_rates = {
                "Alabama": 0.05,
                "California": 0.22,
                # ...
            }
            return state_rates[self.state]  # US sales tax

        elif self.country == "EU":
            return 0.20  # European VAT

        elif self.country == "China":
            product_rates = defaultdict(
                lambda _: 0.2,
                {
                    "Hardware": 0.05,
                    "Food": 0.1,
                    # ...
                },
            )
            return max([product_rates[product.category] for product in self.line_items])

        else:
            return 0


In [11]:
# SOLUTION

from collections import defaultdict
from dataclasses import dataclass


@dataclass
class Item:
    price: int
    quantity: int
    category: str


class TaxCalculator:
    @classmethod
    def get_tax_rate(cls, country, state, products):
        if country == "US":
            return cls._get_us_tax(state)

        elif country == "EU":
            return cls._get_eu_tax(country)

        elif country == "China":
            return cls._get_chinese_tax(products)

        else:
            return 0

    def _get_us_tax(state):
        state_rates = {
            "Alabama": 0.05,
            "California": 0.22,
            # ...
        }
        return state_rates[state]  # US sales tax

    def _get_eu_tax(country):
        return 0.20  # European VAT

    def _get_chinese_tax(products):
        product_rates = defaultdict(
            lambda _: 0.2,
            {
                "Hardware": 0.05,
                "Food": 0.1,
                # ...
            },
        )
        return max([product_rates[product] for product in products])


@dataclass
class Order:
    line_items: list[Item]
    country: str
    state: str = None

    def get_order_total(self):
        total = sum([item.price * item.quantity for item in self.line_items])
        total += total * TaxCalculator.get_tax_rate(
            self.country, self.state, [item.category for item in self.line_items]
        )
        return total


In [12]:
# Unit Test
items = [Item(20, 10, "Food"), Item(1, 100, "Hardware")]
order1 = Order(items, "US", "California")
order2 = Order(items, "EU")
order3 = Order(items, "China")

assert order1.get_order_total() == 366
assert order2.get_order_total() == 360
assert order3.get_order_total() == 330

## Program to an Interface, Not an Implementation

> Program to an interface, not an implementation. Depend on abstractions, not on concrete classes.

### Simple Example

In [2]:
from dataclasses import dataclass

@dataclass
class Sausage:
    nutrition: int
    color: str
    expiration: str

@dataclass
class Cat:
    energy: int

    def eat(self, sausage: Sausage):
        self.energy += sausage.nutrition

In [10]:
# SOLUTION
from abc import ABC, abstractmethod
from dataclasses import dataclass


class Food(ABC):
    @property
    @abstractmethod
    def nutrition(self) -> int:
        ...


@dataclass
class Sausage:
    nutrition_value: int
    color: str
    expiration: str

    @property
    def nutrition(self):
        return self.nutrition_value


@dataclass
class Cat:
    energy: int

    def eat(self, s: Food):
        self.energy += s.nutrition


In [11]:
cat = Cat(10)
sausage = Sausage(5, "brownish", "yesterday")

cat.eat(sausage)
assert cat.energy == 15

### More complex example

In [2]:
# Base scenario
class Designer:
    def design_architecture(self):
        print("Design the architecture ...")


class Programmer:
    def write_code(self):
        print("Write the code ...")


class Tester:
    def test_software(self):
        print("Test the software ...")


class Company:
    def create_software(self):
        designer = Designer()
        designer.design_architecture()
        programmer = Programmer()
        programmer.write_code()
        tester = Tester()
        tester.test_software()


In [4]:
# BETTER: Polymorphism
from abc import ABC, abstractmethod


class Employee(ABC):
    @abstractmethod
    def do_work(self):
        ...


class Designer(Employee):
    def do_work(self):
        print("Design the architecture ...")


class Programmer(Employee):
    def do_work(self):
        print("Write the code ...")


class Tester(Employee):
    def do_work(self):
        print("Test the software ...")


class Company:
    def create_software(self):
        employees = [
            Designer(),
            Programmer(),
            Tester(),
        ]
        for employee in employees:
            employee.do_work()


In [6]:
# Abstract Company
from abc import ABC, abstractmethod


class Employee(ABC):
    @abstractmethod
    def do_work(self):
        ...


class Designer(Employee):
    def do_work(self):
        print("Design the architecture ...")


class Programmer(Employee):
    def do_work(self):
        print("Write the code ...")


class Tester(Employee):
    def do_work(self):
        print("Test the software ...")


class Artist(Employee):
    def do_work(self):
        print("Produce quality art ...")


class Company(ABC):
    @property
    @abstractmethod
    def employees(self) -> list[Employee]:
        ...

    def create_software(self):
        for employee in self.employees:
            employee.do_work()


class GameDevCompany(Company):
    @property
    def employees(self):
        return [
            Designer(),
            Artist(),
            # ...
        ]


class OutSourcingCompany(Company):
    @property
    def employees(self):
        return [
            Programmer(),
            Tester(),
            # ...
        ]


In [8]:
my_company = OutSourcingCompany()
my_company.create_software()

Write the code ...
Test the software ...
